In [77]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(model="meta-llama/Llama-4-Scout-17B-16E-Instruct")

# if the outputs for next cells are wrong, the free model may be overloaded. You can also use this public endpoint that contains Llama-3.2-3B-Instruct
# client = InferenceClient("https://jc26mwg228mkj8dw.us-east-1.aws.endpoints.huggingface.cloud")

In [78]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"},
    ],
    stream=False,
    max_tokens=1024
)
print(output.choices[0].message.content)

Paris.


In [ ]:
# This system prompt is a bit more complex and actually contains the function description already appended.
# Here we suppose that the textual description of the tools has already been appended.

SYSTEM_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have an `action` key (with the name of the tool to use) and an `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}
example use :

{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}


ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:

$JSON_BLOB (inside markdown cell)

Observation: the result of the action. This Observation is unique, complete, and the source of truth.
(this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at a time.)

You must always end your output with the following format:

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. """


[{'role': 'system', 'content': 'Answer the following questions as best you can. You have access to the following tools:\n\nget_weather: Get the current weather in a given location\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have an `action` key (with the name of the tool to use) and an `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are:\nget_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}\nexample use :\n\n{{\n  "action": "get_weather",\n  "action_input": {"location": "New York"}\n}}\n\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about one action to take. Only one action at a time in this format:\nAction:\n\n$JSON_BLOB (inside markdown cell)\n\nObservation: the result of the action. This Observation is unique, complete, and the source of truth.\n(this Thou

In [ ]:
def get_weather(location):
    return f"the weather in {location} is sunny with low temperatures. \n"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What's the weather in London?"},
    {"role": "system", "content": output.choices[0].message.content + get_weather('London')},
]

output = client.chat.completions.create(
    messages=messages,
    max_tokens=200,
    stream=False
)

print(output.choices[0].message.content)

Thought: I now know the final answer

Final Answer: The current weather in London is sunny with low temperatures.
